In [ ]:
!pip install --quiet \
    fastai==2.5.2

In [ ]:
from fastai.vision import *
from fastai.vision.all import * 
from fastai.learner import *
from fastai.learner import load_learner
import pandas as pd
import numpy as np
import PIL
import matplotlib.pyplot as plt
from datetime import datetime

# Env Vars inc YAML Config

In [ ]:
# Import variables from `room-quality-config.yaml`. Note: `room-quality-config.yaml`` must be in the same folder as the notebook you are running
config = yaml.safe_load(open("food-classification-config.yaml")) # load yaml into dictionary

In [ ]:
config

In [ ]:
# create our variables from the yaml config

ROOT_DIR = config.get('root_dir')
BATCH_SIZE = config.get('batch_size')
RANDOM_SEED = config.get('random_seed')
MODEL_DIR = config.get('model_dir')
MODEL_FILE_NAME = config.get('model_file_name')
INPUT_IMAGES_FOLDER = config.get('input_images_folder')
IMAGE_RESIZE_WIDTH = config.get('image_resize_width')
IMAGE_RESIZE_HEIGHT = config.get('image_resize_height')
RESIZED_IMAGE_DIR = f'{INPUT_IMAGES_FOLDER}/resized-{IMAGE_RESIZE_WIDTH}x{IMAGE_RESIZE_HEIGHT}'

In [ ]:
# make our resized image directory

!mkdir -p {RESIZED_IMAGE_DIR}

In [ ]:
# timestamp for versioning model outputs

TIMESTAMP = datetime.now().strftime("%Y%m%d_%H%M")
print(TIMESTAMP)

### Get Images for Predictions

In [ ]:
# get list of images to score

input_imagery = get_image_files(INPUT_IMAGES_FOLDER)

In [ ]:
def resize_images(test_images):
    for i in input_imagery:
        image_open = PIL.Image.open(i)
        width, height = image_open.size
        if width != IMAGE_RESIZE_WIDTH or height != IMAGE_RESIZE_HEIGHT:
            resized_image = image_open.resize((IMAGE_RESIZE_WIDTH, IMAGE_RESIZE_HEIGHT), 0)
            resized_path = f'{RESIZED_IMAGE_DIR}/{i.name}'
            resized_image.convert('RGB').save(resized_path, format='JPEG')

In [ ]:
resize_images(input_imagery)

In [ ]:
# get paths of resized images for feeding into model

resized_imagery = get_image_files(RESIZED_IMAGE_DIR)

In [ ]:
resized_imagery[:5]

### Predict on single image to test

In [ ]:
# load our pkl file

food_classifier_learn = load_learner(f"{MODEL_DIR}/{MODEL_FILE_NAME}")

In [ ]:
EXAMPLE_IMAGE = f'{RESIZED_IMAGE_DIR}/{input_imagery[0].name}'
print(EXAMPLE_IMAGE)

In [ ]:
plt.imshow(np.array(PIL.Image.open(EXAMPLE_IMAGE)))

In [ ]:
food_classifier_learn.predict(EXAMPLE_IMAGE)

### Predict room type of all images in ROOM_TYPE_RESIZED_DIR

In [ ]:
# load data for predictions from the resized_imagery paths
image_filename_pattern = r'/.*(jpg)$'
food_classifier_dls = ImageDataLoaders.from_path_re(path='', 
                                               fnames=resized_imagery, 
                                               pat=image_filename_pattern, 
                                               bs=BATCH_SIZE, 
                                               valid_pct=0, 
                                               shuffle=False)  # see comment two cells down for valid_pct=0 rationale

In [ ]:
# check number of images, validation should be 0
len(food_classifier_dls.train_ds), len(food_classifier_dls.valid_ds)

In [ ]:
# get our predictions
food_classifier_dl = food_classifier_learn.dls.test_dl(food_classifier_dls[0].items)
food_classifier_preds = food_classifier_learn.get_preds(dl=food_classifier_dl, with_decoded=True, reorder=False)

In [ ]:
# recreate list of image paths - can't use resized_imagery here as order may have changed in predictions
image_paths = [str(path) for path in food_classifier_dl.items]

In [ ]:
# extract prediction with probability from predictions output
food_classifier_label_indices = food_classifier_preds[2].tolist()
food_classifier_labels = [food_classifier_learn.dls.vocab[idx] for idx in food_classifier_label_indices]
food_classifier_images = food_classifier_preds[0].tolist()
food_classifier_probs = [probs_for_one_image[label_idx] for probs_for_one_image, label_idx in zip(food_classifier_images, food_classifier_label_indices)]

In [ ]:
# build table of images & predictions
image_preds = pd.DataFrame({"image": image_paths, "pred": food_classifier_labels, "prob": food_classifier_probs})

In [ ]:
# check out our preds!
image_preds